- http://localhost:8888/notebooks/data/git/blnd/nbs/DevCropResize0703.ipynb
- https://www.kaggle.com/c/aptos2019-blindness-detection/discussion/98059#latest-566637
- https://www.kaggle.com/tanlikesmath/diabetic-retinopathy-resnet50-binary-cropped

In [21]:
PRFX='Prep0703'
N_JOBS_PREP = 8
p_o = f'../output/{PRFX}'

# setup

In [17]:
from pathlib import Path
import cv2
import math
import numpy as np
from PIL import Image

from joblib import Parallel, delayed


In [6]:
Path(p_o).mkdir(exist_ok=True, parents=True)
Path(f'{p_o}/aptos2019-blindness-detection/train_images').mkdir(exist_ok=True, parents=True)
Path(f'{p_o}/aptos2019-blindness-detection/test_images').mkdir(exist_ok=True, parents=True)
Path(f'{p_o}/diabetic-retinopathy-detection/train_images').mkdir(exist_ok=True, parents=True)
Path(f'{p_o}/diabetic-retinopathy-detection/test_images').mkdir(exist_ok=True, parents=True)
Path(f'{p_o}/IDRID/B. Disease Grading/1. Original Images/a. Training Set').mkdir(exist_ok=True, parents=True)
Path(f'{p_o}/IDRID/B. Disease Grading/1. Original Images/b. Testing Set').mkdir(exist_ok=True, parents=True)

In [8]:
!find $p_o

../output/Prep0703
../output/Prep0703/aptos2019-blindness-detection
../output/Prep0703/aptos2019-blindness-detection/test_images
../output/Prep0703/aptos2019-blindness-detection/train_images
../output/Prep0703/diabetic-retinopathy-detection
../output/Prep0703/diabetic-retinopathy-detection/test_images
../output/Prep0703/diabetic-retinopathy-detection/train_images
../output/Prep0703/IDRID
../output/Prep0703/IDRID/B. Disease Grading
../output/Prep0703/IDRID/B. Disease Grading/1. Original Images
../output/Prep0703/IDRID/B. Disease Grading/1. Original Images/b. Testing Set
../output/Prep0703/IDRID/B. Disease Grading/1. Original Images/a. Training Set


# prep function

In [22]:
# https://www.kaggle.com/tanlikesmath/diabetic-retinopathy-resnet50-binary-cropped
def crop_image(image, keep_less=0.83):
    output = image.copy()
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    ret,gray = cv2.threshold(gray,10,255,cv2.THRESH_BINARY)
    contours,hierarchy = cv2.findContours(gray,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        #print('no contours!')
        flag = 0
        return image, flag
    cnt = max(contours, key=cv2.contourArea)
    ((x, y), r) = cv2.minEnclosingCircle(cnt)
    r = r*keep_less
    x = int(x); y = int(y); r = int(r)
    flag = 1
    #print(x,y,r)
    if r > 100:
        return output[0 + (y-r)*int(r<y):-1 + (y+r+1)*int(r<y),0 + (x-r)*int(r<x):-1 + (x+r+1)*int(r<x)], flag
    else:
        #print('none!')
        flag = 0
        return image,flag

# https://stackoverflow.com/questions/16646183/crop-an-image-in-the-centre-using-pil
def center_crop(img, new_width=None, new_height=None):        

    width = img.shape[1]
    height = img.shape[0]

    if new_width is None:
        new_width = min(width, height)

    if new_height is None:
        new_height = min(width, height)

    left = int(np.ceil((width - new_width) / 2))
    right = width - int(np.floor((width - new_width) / 2))

    top = int(np.ceil((height - new_height) / 2))
    bottom = height - int(np.floor((height - new_height) / 2))

    if len(img.shape) == 2:
        center_cropped_img = img[top:bottom, left:right]
    else:
        center_cropped_img = img[top:bottom, left:right, ...]

    return center_cropped_img

def process_fnm(fnm):
    fnm = str(fnm)
    h0, w0 = 480, 640 #most common in test
    ratio = h0/w0 #most common in test
    img = cv2.imread(fnm)
    
    img_cropped, flag = crop_image(img)
    img_cropped = cv2.cvtColor(img_cropped, cv2.COLOR_BGR2RGB)

        
    height, width, _= img_cropped.shape
    w_new, h_new = width, math.ceil(width*ratio)
    
    img_cropped_center = center_crop(img_cropped, new_width=w_new, new_height=h_new)
    
    fnm_new = fnm.replace('../input', p_o)
    Image.fromarray(img_cropped_center).save(fnm_new)

# https://www.kaggle.com/c/diabetic-retinopathy-detection/discussion/12803#latest-66496
def process_fnms(fnms, n_jobs):
    Parallel(n_jobs, verbose=10)(delayed(process_fnm)(fnm) for fnm in fnms)

# run prep

## aptos2019-blindness-detection

In [ ]:
p = '../input/aptos2019-blindness-detection'

In [26]:
fnms = list((Path(p)/'train_images').iterdir())

In [27]:
%%time
process_fnms(fnms, n_jobs=N_JOBS_PREP)

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    3.5s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:    5.3s
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    6.9s
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    9.1s
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:   11.1s
[Parallel(n_jobs=8)]: Done  45 tasks      | elapsed:   12.4s
[Parallel(n_jobs=8)]: Done  56 tasks      | elapsed:   14.4s
[Parallel(n_jobs=8)]: Done  69 tasks      | elapsed:   16.8s
[Parallel(n_jobs=8)]: Done  82 tasks      | elapsed:   20.2s
[Parallel(n_jobs=8)]: Done  97 tasks      | elapsed:   23.2s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:   26.6s
[Parallel(n_jobs=8)]: Done 129 tasks      | elapsed:   30.7s
[Parallel(n_jobs=8)]: Done 146 tasks      | elapsed:   34.5s
[Parallel(n_jobs=8)]: Done 165 tasks      | elapsed:   38.3s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:   43.0s
[Parallel(

CPU times: user 22.4 s, sys: 515 ms, total: 22.9 s
Wall time: 13min


[Parallel(n_jobs=8)]: Done 3662 out of 3662 | elapsed: 13.0min finished


In [28]:
fnms = list((Path(p)/'test_images').iterdir())

In [29]:
%%time
process_fnms(fnms, n_jobs=N_JOBS_PREP)

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Batch computation too fast (0.1147s.) Setting batch_size=2.
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done  20 tasks      | elapsed:    0.9s
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    2.5s
[Parallel(n_jobs=8)]: Done  52 tasks      | elapsed:    3.1s
[Parallel(n_jobs=8)]: Done  74 tasks      | elapsed:    4.2s
[Parallel(n_jobs=8)]: Batch computation too slow (2.0620s.) Setting batch_size=1.
[Parallel(n_jobs=8)]: Done  95 tasks      | elapsed:    4.9s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    7.1s
[Parallel(n_jobs=8)]: Done 126 tasks      | elapsed:    8.3s
[Parallel(n_jobs=8)]: Done 141 tasks      | elapsed:    8.9s
[Parallel(n_jobs=8)]: Done 156 tasks      | elapsed:   10.6s
[Parallel(n_jobs=8)]: Done 173 tasks      | elapsed:   12.5s
[Parallel(n_jobs=8)]: Done 19

CPU times: user 11.1 s, sys: 242 ms, total: 11.3 s
Wall time: 2min 28s


[Parallel(n_jobs=8)]: Done 1928 out of 1928 | elapsed:  2.5min finished


## diabetic-retinopathy-detection

In [36]:
p = '../input/diabetic-retinopathy-detection'
fnms = list((Path(p)/'train_images').iterdir())

In [37]:
%%time
process_fnms(fnms, n_jobs=N_JOBS_PREP)

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    0.6s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:    0.8s
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    1.4s
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    2.0s
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    2.7s
[Parallel(n_jobs=8)]: Done  45 tasks      | elapsed:    3.8s
[Parallel(n_jobs=8)]: Done  56 tasks      | elapsed:    4.7s
[Parallel(n_jobs=8)]: Done  69 tasks      | elapsed:    5.7s
[Parallel(n_jobs=8)]: Done  82 tasks      | elapsed:    7.0s
[Parallel(n_jobs=8)]: Done  97 tasks      | elapsed:    8.7s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:   10.0s
[Parallel(n_jobs=8)]: Done 129 tasks      | elapsed:   11.6s
[Parallel(n_jobs=8)]: Done 146 tasks      | elapsed:   13.5s
[Parallel(n_jobs=8)]: Done 165 tasks      | elapsed:   15.1s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:   16.7s
[Parallel(

[Parallel(n_jobs=8)]: Done 8962 tasks      | elapsed: 14.0min
[Parallel(n_jobs=8)]: Done 9097 tasks      | elapsed: 14.2min
[Parallel(n_jobs=8)]: Done 9232 tasks      | elapsed: 14.4min
[Parallel(n_jobs=8)]: Done 9369 tasks      | elapsed: 14.6min
[Parallel(n_jobs=8)]: Done 9506 tasks      | elapsed: 14.8min
[Parallel(n_jobs=8)]: Done 9645 tasks      | elapsed: 15.0min
[Parallel(n_jobs=8)]: Done 9784 tasks      | elapsed: 15.3min
[Parallel(n_jobs=8)]: Done 9925 tasks      | elapsed: 15.5min
[Parallel(n_jobs=8)]: Done 10066 tasks      | elapsed: 15.7min
[Parallel(n_jobs=8)]: Done 10209 tasks      | elapsed: 16.0min
[Parallel(n_jobs=8)]: Done 10352 tasks      | elapsed: 16.2min
[Parallel(n_jobs=8)]: Done 10497 tasks      | elapsed: 16.4min
[Parallel(n_jobs=8)]: Done 10642 tasks      | elapsed: 16.7min
[Parallel(n_jobs=8)]: Done 10789 tasks      | elapsed: 16.9min
[Parallel(n_jobs=8)]: Done 10936 tasks      | elapsed: 17.1min
[Parallel(n_jobs=8)]: Done 11085 tasks      | elapsed: 17.4min


[Parallel(n_jobs=8)]: Done 35097 tasks      | elapsed: 55.1min


CPU times: user 3min 15s, sys: 4.62 s, total: 3min 20s
Wall time: 55min 7s


[Parallel(n_jobs=8)]: Done 35126 out of 35126 | elapsed: 55.1min finished


In [38]:
fnms = list((Path(p)/'test_images').iterdir())

In [39]:
%%time
process_fnms(fnms, n_jobs=N_JOBS_PREP)

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    0.5s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:    1.0s
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    1.3s
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    2.3s
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    3.3s
[Parallel(n_jobs=8)]: Done  45 tasks      | elapsed:    4.2s
[Parallel(n_jobs=8)]: Done  56 tasks      | elapsed:    5.8s
[Parallel(n_jobs=8)]: Done  69 tasks      | elapsed:    7.0s
[Parallel(n_jobs=8)]: Done  82 tasks      | elapsed:    8.1s
[Parallel(n_jobs=8)]: Done  97 tasks      | elapsed:    9.6s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:   11.0s
[Parallel(n_jobs=8)]: Done 129 tasks      | elapsed:   12.5s
[Parallel(n_jobs=8)]: Done 146 tasks      | elapsed:   14.4s
[Parallel(n_jobs=8)]: Done 165 tasks      | elapsed:   16.2s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:   18.6s
[Parallel(

[Parallel(n_jobs=8)]: Done 9506 tasks      | elapsed: 14.9min
[Parallel(n_jobs=8)]: Done 9645 tasks      | elapsed: 15.1min
[Parallel(n_jobs=8)]: Done 9784 tasks      | elapsed: 15.4min
[Parallel(n_jobs=8)]: Done 9925 tasks      | elapsed: 15.6min
[Parallel(n_jobs=8)]: Done 10066 tasks      | elapsed: 15.9min
[Parallel(n_jobs=8)]: Done 10209 tasks      | elapsed: 16.1min
[Parallel(n_jobs=8)]: Done 10352 tasks      | elapsed: 16.3min
[Parallel(n_jobs=8)]: Done 10497 tasks      | elapsed: 16.6min
[Parallel(n_jobs=8)]: Done 10642 tasks      | elapsed: 16.8min
[Parallel(n_jobs=8)]: Done 10789 tasks      | elapsed: 17.0min
[Parallel(n_jobs=8)]: Done 10936 tasks      | elapsed: 17.2min
[Parallel(n_jobs=8)]: Done 11085 tasks      | elapsed: 17.5min
[Parallel(n_jobs=8)]: Done 11234 tasks      | elapsed: 17.7min
[Parallel(n_jobs=8)]: Done 11385 tasks      | elapsed: 17.9min
[Parallel(n_jobs=8)]: Done 11536 tasks      | elapsed: 18.2min
[Parallel(n_jobs=8)]: Done 11689 tasks      | elapsed: 18.4

[Parallel(n_jobs=8)]: Done 36165 tasks      | elapsed: 57.1min
[Parallel(n_jobs=8)]: Done 36434 tasks      | elapsed: 57.6min
[Parallel(n_jobs=8)]: Done 36705 tasks      | elapsed: 58.0min
[Parallel(n_jobs=8)]: Done 36976 tasks      | elapsed: 58.4min
[Parallel(n_jobs=8)]: Done 37249 tasks      | elapsed: 58.9min
[Parallel(n_jobs=8)]: Done 37522 tasks      | elapsed: 59.4min
[Parallel(n_jobs=8)]: Done 37797 tasks      | elapsed: 59.8min
[Parallel(n_jobs=8)]: Done 38072 tasks      | elapsed: 60.2min
[Parallel(n_jobs=8)]: Done 38349 tasks      | elapsed: 60.7min
[Parallel(n_jobs=8)]: Done 38626 tasks      | elapsed: 61.1min
[Parallel(n_jobs=8)]: Done 38905 tasks      | elapsed: 61.6min
[Parallel(n_jobs=8)]: Done 39184 tasks      | elapsed: 62.1min
[Parallel(n_jobs=8)]: Done 39465 tasks      | elapsed: 62.5min
[Parallel(n_jobs=8)]: Done 39746 tasks      | elapsed: 62.9min
[Parallel(n_jobs=8)]: Done 40029 tasks      | elapsed: 63.4min
[Parallel(n_jobs=8)]: Done 40312 tasks      | elapsed: 

CPU times: user 4min 57s, sys: 7.47 s, total: 5min 4s
Wall time: 1h 24min 43s


[Parallel(n_jobs=8)]: Done 53576 out of 53576 | elapsed: 84.7min finished


## IDRID

In [31]:
p = '../input/IDRID/B. Disease Grading/1. Original Images'

In [32]:
fnms = list((Path(p)/'a. Training Set').iterdir())

In [33]:
%%time
process_fnms(fnms, n_jobs=N_JOBS_PREP)

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    0.8s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:    1.6s
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    2.1s
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    3.4s
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    4.3s
[Parallel(n_jobs=8)]: Done  45 tasks      | elapsed:    5.4s
[Parallel(n_jobs=8)]: Done  56 tasks      | elapsed:    6.8s
[Parallel(n_jobs=8)]: Done  69 tasks      | elapsed:    8.0s
[Parallel(n_jobs=8)]: Done  82 tasks      | elapsed:    9.8s
[Parallel(n_jobs=8)]: Done  97 tasks      | elapsed:   11.4s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:   13.1s
[Parallel(n_jobs=8)]: Done 129 tasks      | elapsed:   14.9s
[Parallel(n_jobs=8)]: Done 146 tasks      | elapsed:   16.7s
[Parallel(n_jobs=8)]: Done 165 tasks      | elapsed:   19.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:   21.0s
[Parallel(

CPU times: user 2.35 s, sys: 104 ms, total: 2.45 s
Wall time: 46.2 s


[Parallel(n_jobs=8)]: Done 413 out of 413 | elapsed:   46.2s finished


In [34]:
fnms = list((Path(p)/'b. Testing Set').iterdir())

In [35]:
%%time
process_fnms(fnms, n_jobs=N_JOBS_PREP)

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    3.7s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:    5.0s
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    6.0s
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    7.1s
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    8.2s
[Parallel(n_jobs=8)]: Done  45 tasks      | elapsed:    9.6s
[Parallel(n_jobs=8)]: Done  56 tasks      | elapsed:   11.1s
[Parallel(n_jobs=8)]: Done  69 tasks      | elapsed:   12.6s
[Parallel(n_jobs=8)]: Done  82 tasks      | elapsed:   14.0s
[Parallel(n_jobs=8)]: Done  99 out of 103 | elapsed:   15.9s remaining:    0.6s


CPU times: user 608 ms, sys: 78.9 ms, total: 687 ms
Wall time: 16.5 s


[Parallel(n_jobs=8)]: Done 103 out of 103 | elapsed:   16.5s finished


# see result

In [30]:
!find $p_o

../output/Prep0703
../output/Prep0703/aptos2019-blindness-detection
../output/Prep0703/aptos2019-blindness-detection/test_images
../output/Prep0703/aptos2019-blindness-detection/test_images/95c3558dc279.png
../output/Prep0703/aptos2019-blindness-detection/test_images/464ee4a542b0.png
../output/Prep0703/aptos2019-blindness-detection/test_images/01499815e469.png
../output/Prep0703/aptos2019-blindness-detection/test_images/8bdd3a1895c8.png
../output/Prep0703/aptos2019-blindness-detection/test_images/cf95d13f210a.png
../output/Prep0703/aptos2019-blindness-detection/test_images/e03bb55dd03c.png
../output/Prep0703/aptos2019-blindness-detection/test_images/75c6563ed058.png
../output/Prep0703/aptos2019-blindness-detection/test_images/7007d3ba7e79.png
../output/Prep0703/aptos2019-blindness-detection/test_images/03042a663e54.png
../output/Prep0703/aptos2019-blindness-detection/test_images/95dafd1d5f47.png
../output/Prep0703/aptos2019-blindness-detection/test_images/3832a69f943a.png
../output/Pre

../output/Prep0703/aptos2019-blindness-detection/train_images/61f403fdb434.png
../output/Prep0703/aptos2019-blindness-detection/train_images/349f3c0ac83e.png
../output/Prep0703/aptos2019-blindness-detection/train_images/bfefa7344e7d.png
../output/Prep0703/aptos2019-blindness-detection/train_images/e229aca862c7.png
../output/Prep0703/aptos2019-blindness-detection/train_images/4818672273af.png
../output/Prep0703/aptos2019-blindness-detection/train_images/ef26625121b3.png
../output/Prep0703/aptos2019-blindness-detection/train_images/dc6fa1b38b83.png
../output/Prep0703/aptos2019-blindness-detection/train_images/df4913ca3712.png
../output/Prep0703/aptos2019-blindness-detection/train_images/63c0eafd6aa9.png
../output/Prep0703/aptos2019-blindness-detection/train_images/f48241b0c995.png
../output/Prep0703/aptos2019-blindness-detection/train_images/5445255635f0.png
../output/Prep0703/aptos2019-blindness-detection/train_images/88e5051f65bd.png
../output/Prep0703/aptos2019-blindness-detection/tra